# Center-TBI statistics (from full dump via XNAT REST API/pyxnat)
Compute statistics from the full database dumped in JSON from XNAT REST API (via pyxnat). Please generate beforehand `xnat_data_extrat.json` by executing `xnat_data_extractor.ipynb` (or get the file somewhere else, since it contains private project data it cannot be stored publicly here).

Please edit login.cfg with your credentials before executing this script.

Before (re-)running this script, please clear output, shutdown and relaunch kernel, close down and reopen your browser, and then (re-)launch all the cells! Else the memory is not correctly freed (this is a bug in ipywidgets or jupyter notebook).


In [ ]:
%load_ext autoreload
%autoreload 2
# BEWARE: autoreload works on functions and on general code, but NOT on new class methods:
# if you add or change the name of a method, you have to reload the kernel!
# also it will fail if you use super() calls in the classes you change

# Profilers:
# http://pynash.org/2013/03/06/timing-and-profiling/
# http://mortada.net/easily-profile-python-code-in-jupyter.html
# use %lprun -m module func(*args, **kwargs)
try:
    %load_ext line_profiler
    %load_ext memory_profiler
    from fdict import fdict
except ImportError as exc:
    pass

import json
import pandas as pd

# Setup some display options for pandas
pd.set_option('max_columns', 400)
pd.set_option('expand_frame_repr', False)

## Parameters

In [ ]:
# Edit the filepath to the json file here
json_filepath = 'xnat_data_extract.json'

## Auxiliary Functions

In [ ]:
def json_load_dict(json_filepath):
    with open(json_filepath, 'rb') as f:
        #j = f.read()
        jdict = json.load(f)
        #del j
    return jdict

## Extracting pertinent database fields into a Dataframe
The original structure is very nested, here we extract only the relevant fields to get a 2D matrix of only the relevant fields.

In [ ]:
# Load the json dict
jdict = json_load_dict(json_filepath)

In [ ]:
# Reorder the json dict so that it is organized per scan session instead of per project
def ensure_list(obj):
    '''Ensure an object is a collection/list, and not a single object (this simplifies looping)'''
    if '@ID' in obj:
        # If it has an id attribute, this is a single object, we need to transform it into a list
        obj = {obj['@ID']: obj}  # we use dicts here instead of lists (for various technical reasons)
    return obj

def copy_dict_exclude(d, exclude):
    '''Copy a dict but exclude the specified keys'''
    return {k:v for k,v in d.items() if k not in exclude}

exclude_resources = False  # exclude resources because they use up a lot of columns and are pretty much useless for stats (only useful for download)

# Initialization, do not touch
scans_excludes = ['@xsi:schemaLocation']
if exclude_resources:
    scans_excludes.append('resources')

# Create a dict per scan
jdict2 = {'scans': []}
for projectname, project in ensure_list(jdict['projects']).items():
    # For each project
    for subjectname, subject in ensure_list(project['subjects']).items():
        # For each subject
        for experimentname, experiment in ensure_list(subject['experiments']).items():
            # For each experiment
            for scanname, scan in ensure_list(experiment['scans']).items():
                # For each scan
                # Get the scan infos (excluding the columns/fields we do not want)
                scan_temp = copy_dict_exclude(scan, scans_excludes)
                if 'resources' in scan_temp:
                    # Convert to a list so the list of files does not get flattened as columns
                    if 'SNAPSHOTS' in scan_temp['resources'] and 'files' in scan_temp['resources']['SNAPSHOTS']:
                        scan_temp['resources']['SNAPSHOTS']['files'] = [scan_temp['resources']['SNAPSHOTS']['files'].copy()]
                    if 'files' in scan_temp['resources']:
                        scan_temp['resources']['files'] = [scan_temp['resources']['files'].copy()]
                # Add the scan infos
                jdict2['scans'].append(copy_dict_exclude(scan, scans_excludes))  # add this scan record
                currecord = jdict2['scans'][-1]  # keep it at hand so we can add infos of above levels
                # Add the experiment infos for this scan (these infos are shared across all scans of this experiment, so we duplicate for each scan)
                currecord['experiment'] = copy_dict_exclude(experiment, ['scans', 'xnat:scans', '@xsi:schemaLocation'])
                # Same for subject infos
                currecord['subject'] = copy_dict_exclude(subject, ['experiments', 'xnat:experiments', '@xsi:schemaLocation'])
                # Same for project infos
                currecord['project'] = copy_dict_exclude(project, ['subjects', 'xnat:subjects', '@xsi:schemaLocation'])
                # Compate agregative score for QA fields
                try:
                    a = json.loads(currecord['experiment']['xnat:assessors']['xnat:assessor']['ext:results_dict'], strict=False)  # need strict=False because of non-escaped \n and \r (line return + carriage return)
                    currecord['qa.protocol_check'] = a['protocol_check']
                    currecord['qa.head_coverage'] = [[k,v] for k,v in a['head_coverage'].items()]
                    currecord['qa.protocol_check_global'] = all(1 if v.lower() == 'pass' else 0 for _, v in a['protocol_check'])
                    currecord['qa.head_coverage_global'] = all(1 if v.lower() == 'good' else 0 for v in a['head_coverage'].values())
                except Exception:
                    pass


In [ ]:
del jdict

In [ ]:
# Postprocessing: we convert nested dicts into json strings
#for scanid in xrange(len(jdict2['scans'])):
#    for key, value in jdict2['scans'][scanid].items():
#        if isinstance(value, dict):
#            for subkey, subvalue in jdict2['scans'][scanid][key].items():
#                if isinstance(subvalue, dict):
#                    jdict2['scans'][scanid][key][subkey] = json.dumps(subvalue)

In [ ]:
# You should check these files to see if the structure is correct and has all required fields

# Save one record for test
with open('center-tbi-statistics-from-full-rest-dump_onerecordtest.json', 'w') as f:
    json.dump(currecord, f, indent=4, sort_keys=True)
# Save all reordered records
with open('center-tbi-statistics-from-full-rest-dump_persession.json', 'w') as f:
    json.dump(jdict2, f, indent=4, sort_keys=True)

In [ ]:
# Flatten keys so that pandas can access all nested dicts
jdict3 = [fdict.flatkeys(scan, sep='.') for scan in jdict2['scans']]
# Save flattened records
with open('center-tbi-statistics-from-full-rest-dump_persession_flattened.json', 'w') as f:
    json.dump(jdict3, f, indent=4, sort_keys=True)
# Free up memory
del jdict2

In [ ]:
# Convert to a pandas dataframe!
# TODO: if out of memory error, can try to use json-streamer with a mockup-class to return a dict-like generator to pandas: https://github.com/kashifrazzaqui/json-streamer
df = pd.io.json.json_normalize(jdict3)
# Free up memory
del jdict3

In [ ]:
# Show structure of df
print('The dataframe has %s (lines, columns)' % str(df.shape))
print('Columns of the dataframe: ')
print(df.columns)
df  # to pretty print, just put the dataframe name as the last line, without print or anything

In [ ]:
print('Count of unique values per columns:')
for c in df.columns:
    try:
        print('* %s: %s' % (c, str(len(df[c].unique()))))
    except Exception as exc:
        pass

In [ ]:
print('Count per value for all columns:')
for c in df.columns:
    print "---- %s ---" % c
    print df[c].value_counts()

In [ ]:
a = df.apply(lambda df: (df['project.id'], df['qa.protocol_check_global']), axis=1)
print('Number of validated scans per center')
a.value_counts()

In [ ]:
print('Only centers with validated scans:')
a.apply(lambda x: x if x[1] == True else None).dropna().value_counts()

In [ ]:
print('Total number of validated scans:')
len(a.apply(lambda x: x if x[1] == True else None).dropna())

In [ ]:
print('Only centers with unvalidated scans:')
a.apply(lambda x: x if x[1] != True else None).dropna().value_counts()

In [ ]:
#df[['project.id', 'qa.protocol_check_global']].groupby(['project.id']).value_counts()

In [ ]:
a = df.apply(lambda df: (df['project.id'], df['qa.head_coverage_global']), axis=1)
print('Number of validated head coverage scans per center')
a.value_counts()

In [ ]:
phantom_validated_centers = df.where(df['experiment.@visit_id'] == 'Phantom')['project.id'].dropna().unique()
print('List of Phantom validated centers:')
for center in sorted(phantom_validated_centers):
    print('* %s' % center)

In [ ]:
# TODO: filter by resting state fmri
# TODO: make figures of all of these infos (number of validated/unvalidated scans per center, number of fmri validated scans)

### rs_fMRI

In [ ]:
print('Scan types:')
for t in sorted(df['@type'].unique()):
    print('* '+ str(t))

In [ ]:
df['@type'].value_counts().sort_values(ascending=False)

In [ ]:
# Extract only resting state data
df_rest = df.loc[df['@type'] == 'rs_fMRI', :]
df_rest[['@UID', 'project.id', 'qa.protocol_check_global']]

## Stats figures
Generate interesting stats figures.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

In [ ]:
def plot_boolean(df, col1, col2, morecols=None, where=None, groupby=None, truename='validated', falsename='not validated', title=None, nostats=False, nograph=False):
    '''Plot the col2 according to col1, counting for the number of unique values (true/false or true/nan)'''
    # Prepare stuff
    if not morecols:
        morecols = []
    if not groupby:
        groupby = []
    elif isinstance(groupby, str):
        groupby = [groupby]
    # Extract both columns
    if where is not None:
        a = df.loc[where, [col1, col2]+morecols+groupby]
    else:
        a = df[[col1, col2]+morecols+groupby]
    # Group by the first column (we call this column the categories)
    groupby.insert(0, col1)
    b = a.groupby(groupby)
    # Aggregate using the sum (or count of True values) and size (the total number of entries)
    c = b.aggregate(['sum', 'size'])
    # Replace nan/none values by 0 (when there is absolutely no true value for one category)
    c.fillna(0, inplace=True)
    # Because of aggregation we get a hierarchical multiindex, we flatten the indices for easier access
    c.columns = c.columns.get_level_values(1)
    # Compute the number of false values (true values count - total values count)
    c['size'] = c['size'] - c['sum']
    # Rename the columns
    c = c.rename(columns={'sum': truename, 'size': falsename})
    if not nostats:
        # Print the table
        print(c)
        # Print stats
        print(c.describe())
        print('Total:\n'+str(c.sum()))
    if not nograph:
        # Plot as stacked bars
        c.plot.bar(stacked=True, title=title)
        plt.show()
        # Plot only validated
        c[truename].plot.bar(stacked=True, title=(title+' (validated only)'))
        plt.show()

In [ ]:
plot_boolean(df, 'project.id', 'qa.protocol_check_global', title='Protocol check global (all scan types)')

In [ ]:
plot_boolean(df, 'project.id', 'qa.head_coverage_global', truename='Good', falsename='Bad', title='Head coverage global (all scan types)')

In [ ]:
plot_boolean(df, 'project.id', 'qa.head_coverage_global', where=(df['@type'] == 'rs_fMRI'), title='Protocol check global (rs_fMRI)')

In [ ]:
print('Number of subjects with validated rs_fMRI')
plot_boolean(df, 'project.id', 'qa.head_coverage_global', where=(df['@type'] == 'rs_fMRI'), groupby=['subject.@ID'], title='Protocol check global (rs_fMRI)', nograph=True)

--------------------------------
## Resources
Useful resources:
* https://stackoverflow.com/questions/34092808/extract-nested-json-embedded-as-string-in-pandas-dataframe#
* https://stackoverflow.com/questions/39899005/how-to-flatten-a-pandas-dataframe-with-some-columns-as-json
* http://mindtrove.info/flatten-nested-json-with-pandas/

----------------------------------------------
## Random code

In [ ]:
def flatten_json(y):
    # From https://medium.com/@amirziai/flattening-json-objects-in-python-f5343c794b10
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

In [ ]:
def JSON2DICTParser(data):
    # From https://stackoverflow.com/questions/20680272/parsing-a-json-string-which-was-loaded-from-a-csv-using-pandas
    if data and re.sub('[\[\]{}\s]+', '', data):
        try:
            j1 = json.loads(data)
        except ValueError:
            print(data)
            raise
        return j1
    else:
        return float('nan')

# Load the csv into a Pandas DataFrame
#df = pd.read_csv(json_filepath, converters={'software_metadata':JSON2DICTParser, 'parameters_dict':JSON2DICTParser, 'results_dict':JSON2DICTParser})
df = pd.read_csv(json_filepath)
